In [5]:
import random
import csv

class WitnessGrid:
    def __init__(self):
        self.grid = []
        self.head = []

        #we can decide which one we'd like to keep
        self.moves = [] # e.g. up, left, etc.
        self.trail = [] # e.g. [4,0] for the vertex 'b'
    
    def makeBasic2x2(self):
        self.grid = [['v', 'e', 'v', 'e', 'f'], 
                     ['e', ' ', 'e', ' ', 'e'],
                     ['v', 'e', 'v', 'e', 'v'],
                     ['e', ' ', 'e', ' ', 'e'],
                     ['b', 'e', 'v', 'e', 'v']]
        
    def makeBasicNxN(self, n):
        if n > 0:
            self.grid = [['v', 'e', 'v'], 
                         ['e', ' ', 'e'], 
                         ['v', 'e', 'v']]
            for x in range(n - 1):
                self.grid.append(['e', ' ', 'e'])
                self.grid.append(['v', 'e', 'v'])
            for x in range(len(self.grid)):  
                for y in range(n - 1):
                    if x % 2 == 0:
                        self.grid[x].extend(['e', 'v'])
                    else:
                        self.grid[x].extend([' ', 'e'])
            self.grid[0][-1] = 'f'
            self.grid[-1][0] = 'b'
            return self.grid
        else: 
            return 'Choose n >= 1 for grid size!'
        
    # note that when n is odd, a dotted grid may occasionally be unsolvable
    def makeDottedNxN(self, n, percent):
        if n > 0:
            self.grid = [['v', 'e', 'v'], 
                         ['e', ' ', 'e'], 
                         ['v', 'e', 'v']]
            for x in range(n - 1):
                self.grid.append(['e', ' ', 'e'])
                self.grid.append(['v', 'e', 'v'])
            for x in range(len(self.grid)):  
                for y in range(n - 1):
                    if x % 2 == 0:
                        self.grid[x].extend(['e', 'v'])
                    else:
                        self.grid[x].extend([' ', 'e'])
            self.grid[0][-1] = 'f'
            self.grid[-1][0] = 'b'
            
            # add dots
            for x in range(len(self.grid)):
                if x % 2 == 0:
                    for y in range(len(self.grid[x])):
                        if self.grid[x][y] == 'v' and random.randint(1,100) <= percent:
                            self.grid[x][y] = 'X'
            
            return self.grid
        else: 
            return 'Choose n >= 1 for grid size!'
        
    def begin(self):
        self.head = []
        self.trail = []
        self.moves = []
        for row in self.grid:
            for item in row:
                if item == 'b':
                    self.head = [self.grid.index(row), row.index(item)]
                    self.trail.append(self.head)
                    
                    
                    
                    
                
#     def makeBruteForceLogs(self): # assumes you already have a grid and have called begin()
#         # given a starting grid state, try moving u/d/l/r.
#         # for each new state, if it wasn't invalid, do it again until you reach only end states.
#         # keep a log of each path you generated (and also pick the best path to test against?)
#         # evaluate how good a position is with, like, number of steps it takes to win (so some are infinty and bad)
#         # put that into a CNN so we have a model (or decision tree) that gives us value of a sub-path.
#         bruteForceStep(self.trail, self.moves)


#IT WORKS! but I think we should not let it take invalid moves. Maybe. uncomment the line in invalid to see them

    def startBruteForce(self):
        self.head = []
        self.moves = []
        self.trail = []
        self.begin()
        
        self.bruteForceStep('up')
        self.bruteForceStep('right')
        self.bruteForceStep('down')
        self.bruteForceStep('left')
        
        return 'Done!'
        
        # now using the file, or more work in the step function, evaluate ? or make that a different func.
        
    def bruteForceStep(self, direction):
        
        if direction == 'up':
            result = self.moveup()
        elif direction == 'down':
            result = self.movedown()
        elif direction == 'left':
            result = self.moveleft() 
        elif direction == 'right':
            result = self.moveright()
            
        if result[0:7] == 'Invalid':
            pass
#             self.write(self.moves + [direction, 'Invalid'])
        elif result == 'Solved!':
            self.write(self.moves + ['Solved'])
            self.moves.pop()
            self.trail.pop()
        elif result[0:8] == 'Finished':
            self.write(self.moves + ['Finished'])
            self.moves.pop()
            self.trail.pop()
        else:
            self.bruteForceStep('up')
            self.bruteForceStep('right')
            self.bruteForceStep('down')
            self.bruteForceStep('left')
            self.moves.pop()
            self.trail.pop()
            
    def write(self, path): # https://www.geeksforgeeks.org/writing-csv-files-in-python/
        filename = "witnessData.csv"
        with open(filename, 'a') as csvfile: 
            csvwriter = csv.writer(csvfile) 
            csvwriter.writerow(path) 
            
            
            
            
            
    def step(self):
        choice = input("Type u,d,l,r for up/down/left/right: ")
        if choice == 'u':
            return self.moveup()
        elif choice == 'd':
            return self.movedown()
        elif choice == 'l':
            return self.moveleft()
        elif choice == 'r':
            return self.moveright()
        else:
            print("Choose u/d/l/r")
            return
    
    def evaluateState(self, move):
        '''
        This function returns 'Solved' if the current vertex is the final vertex
        otherwise it returns the current trail
        '''
        self.moves.append(move)
        curr_row, curr_col = self.trail[-1] #gets the last row inserted after moving
        
        gotEveryDot = True
        for x in range(len(self.grid)):
            if gotEveryDot == False: break
            if x % 2 == 0:
                for y in range(len(self.grid[x])):
                    if self.grid[x][y] == 'X':
                        if [x, y] not in self.trail:
                            gotEveryDot = False
                            break
        
        if self.grid[curr_row][curr_col] == 'f':
            if gotEveryDot == True:
                return 'Solved!'
            else:
                return 'Finished, but missed one or more dots!'
        else:
            return self.trail

    def getNumRows(self):
        return len(self.grid) - 1
    
    def getNumCols(self):
        return len(self.grid[0]) - 1
    
    def getMoves(self):
        '''Returns the list of moves entered'''
        return self.moves
        
    def moveup(self):
        '''Gets the last position moved and checks if there is an edge
            to the next vertex. If an edge exists, it moves up to the next
            vertex, otherwise it returns and 'invalid' move.
        '''
        row, col = self.trail[-1]
        if row >= 0 and self.grid[row - 1][col] == 'e':
            if self.trail.count([row - 2, col]) > 0:
                return 'Invalid: Crossed Path!'
            self.trail.append([row - 2, col])
            return self.evaluateState('up')
        return 'Invalid: Out of Bounds!'

    def movedown(self):
        '''Gets the last position moved and checks if there is an edge
            to the next vertex. If an edge exists, it moves down to the next
            vertex, otherwise it returns and 'invalid' move.
        '''
        row, col = self.trail[-1]
        if row < self.getNumRows() and self.grid[row + 1][col] == 'e':
            if self.trail.count([row + 2, col]) > 0:
                return 'Invalid: Crossed Path!'
            self.trail.append([row + 2, col])
            return self.evaluateState('down')
        return 'Invalid: Out of Bounds!'
    
    def moveleft(self):
        row, col = self.trail[-1]
        if col >= 0 and self.grid[row][col-1] == 'e':
            if self.trail.count([row, col - 2]) > 0:
                return 'Invalid: Crossed Path!'
            self.trail.append([row, col - 2])
            return self.evaluateState('left')
        return 'Invalid: Out of Bounds!'
    
    def moveright(self):
        row, col = self.trail[-1]
        if col < self.getNumCols() and self.grid[row][col + 1] == 'e':
            if self.trail.count([row, col + 2]) > 0:
                return 'Invalid: Crossed Path!'
            self.trail.append([row, col + 2])
            return self.evaluateState('right')
        return 'Invalid: Out of Bounds!'

In [6]:
# testing out the class manually
test = WitnessGrid()
# test.makeBasic2x2()
# test.makeBasicNxN(3) # make 3x3 grid
# test.makeDottedNxN(2, 50) # make 2x2 grid where ~half of the vertices are 'required'
test.makeDottedNxN(2, 50)

[['X', 'e', 'v', 'e', 'f'],
 ['e', ' ', 'e', ' ', 'e'],
 ['X', 'e', 'v', 'e', 'v'],
 ['e', ' ', 'e', ' ', 'e'],
 ['b', 'e', 'v', 'e', 'X']]

In [3]:
#Feel free to try other scenarios to see how the invalid part works.
test.begin()
step = test.step()
print(step)
while step != 'Invalid':
    step = test.step()
    print(step)
    if step == 'Solved!' or step == 'Finished, but missed one or more dots!':
        break
print(test.getMoves())

Type u,d,l,r for up/down/left/right: r
[[4, 0], [4, 2]]
Type u,d,l,r for up/down/left/right: r
[[4, 0], [4, 2], [4, 4]]
Type u,d,l,r for up/down/left/right: u
[[4, 0], [4, 2], [4, 4], [2, 4]]
Type u,d,l,r for up/down/left/right: u
Solved!
['right', 'right', 'up', 'up']


In [7]:
test.startBruteForce()

'Done!'

Notes:
For 1x1: 2 Solutions
For 2x2: 12 Solutions
For 3x3: 184 Solutions
For 4x4: 8512 Solutions
A 5x5 would take too long to brute force. Two hours at least. -Zach Clark